In [36]:
%reset -f

In [37]:
#!/usr/bin/env python3
import pandas as pd
from tqdm import tqdm
from PIL import Image
import torch
from torchvision import transforms
from torch import nn

In [38]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data_file_path):
        self.data = pd.read_pickle(data_file_path)
        self.image_transform = transforms.Compose([transforms.ToTensor()])


    def __getitem__(self, idx):
        image_path = self.data['path'][idx]
        image = Image.open(image_path)
        label = self.data['label'][idx]
        return image, label
    

    def __len__(self):
        return len(self.data)

In [39]:
class FaceNet(nn.Module):
    pass